In [71]:
#coding:utf-8
import pandas as pd
import numpy as np
import time
import datetime
import gc
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [72]:
path = '/Users/apple/Desktop/data/'
data = pd.read_csv(path+"all_final_data_11-28-12-33.csv")


In [73]:
train = pd.read_csv(path+"train_all.csv")
train = train[['instance_id']]
train =pd.merge(train, data, how='left', on='instance_id')

In [74]:
def feat_select(train):
  
    features = train.drop(['is_trade', 'time'], axis=1).columns.tolist()

    target = ['is_trade']

    return features, target

In [75]:
def lgbCV(train, test):
    features, target = feat_select(train)

    lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.05)
    # lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.05, n_estimators=2000,
    #                            max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
    #                            min_child_weight=5, min_child_samples=10, subsample=1, subsample_freq=1,
    #                            colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, nthread=-1, silent=True)

    lgb_model = lgb_clf.fit(train[features], train[target], eval_set=[(test[features], test[target])])
    best_iter = lgb_model.best_iteration_ 
    
    # 特征重要性
    lgb_predictors = [i for i in train[features].columns]
    lgb_feat_imp = pd.Series(lgb_model.feature_importances_, lgb_predictors).sort_values(ascending=False)
    lgb_feat_imp.to_csv(path+'lgb_feat_imp.csv')
    
    # 训练模型
    lgb_clf.fit(train[features], train[target])
    test['lgb_predict'] = lgb_clf.predict_proba(test[features])[:, 1]
    lgb_loss = log_loss(test[target], test['lgb_predict'])  
    lgb_auc = roc_auc_score(test[target], test['lgb_predict'])
    print('Training loss: %.6f, Training AUC: %.6f' % (lgb_loss, lgb_auc))
    
    return best_iter,lgb_loss

In [76]:
test = pd.read_csv(path+"test_all.csv")
test = test[['instance_id']]
test = pd.merge(test, data, how='left', on='instance_id')

In [77]:
best_iter, lgb_loss = lgbCV(train, test)

/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 1128, number of negative: 78140
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007373 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7020
[LightGBM] [Info] Number of data points in the train set: 79268, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.014230 -> initscore=-4.238056
[LightGBM] [Info] Start training from score -4.238056


/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/apple/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 1128, number of negative: 78140
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005659 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7020
[LightGBM] [Info] Number of data points in the train set: 79268, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.014230 -> initscore=-4.238056
[LightGBM] [Info] Start training from score -4.238056
Training loss: 0.065474, Training AUC: 0.765417


In [78]:
best_iter

0

In [79]:
lgb_loss

0.06547381177173461